In [16]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  4 14:56:42 2016
@author: Artem
"""
from IPython.display import clear_output
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
#Библиотека содержит
import get_data_dic
import distr
from tqdm import tqdm
#Вывод на экран текущего времени (час, мин)
localtime = time.localtime(time.time())
print ("Local current time :", localtime[3:5])
#Ссылкы на первую страницу выдачи при поиске
#квартир из определенного района при поиске квартир из ЦАО
def split_link(link):
    return link.split('/')[-1]

mln_boarders=[]
for mln_price in range(1,50,1):
    mln_boarders.append(mln_price*1000000)
    
parsed_files_list =['output/big_files/data08_01_2017__11_33_36.csv',
                    'output/big_files/data29_01_2017__02_29_45.csv',
                   'output/big_files/data29_01_2017__10_29_57.csv']
parsed_df = []
for parsed_file in parsed_files_list:
    df1 = pd.read_csv(parsed_file,sep=';',encoding='cp1251')
    if len(parsed_df)!=0:
        parsed_df = pd.concat([parsed_df,df1])
    else:
        parsed_df = df1.copy()
parsed_links = list(parsed_df['Link'])
print(parsed_links)
#parsed_links = list(map(split_link,parsed_links))
print(parsed_links[0:10])
dist_list =[]
dist_list_show=[]
min_boarder=0
for boarder in mln_boarders:
    a = 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2'
    b = '&maxprice=' + str(boarder)
    c = '&minprice=' + str(min_boarder+1)
    d = '&offer_type=flat&'
    e = 'p={}'
    f = '&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
    dist_list_show.append(' from ' + str(min_boarder/1000000) + ' to ' + str(boarder/1000000)+' mln')
    min_boarder = boarder
    abcdef = a + b + c + d + e + f
    #print(abcdef)
    dist_list.append(abcdef)
#print(dist_list)
#dist_list=[cent]
#X - будущий большой датасет со всеми квартирами и фичами
N=30 #число страниц, по которым мы бегаем, ища
X=[]#all data
#Цикл по всем районам. Номер района в цикле это и есть номер,
#под которым этот районом будет закодирован в конечном датасете
ID=0 #номер квартиры в датасете
for distN in tqdm(range( len(dist_list) )):
    #print(dist_list_show[distN])
    time1 = time.localtime(time.time())
    #print ("Local current time :", time1[3:5])
    #по главной ссылке находим номера страниц каждой из квартир
    #например, на нулевой итерации links - список уникальных номеров всех
    #квартир из ЦАО
    links=distr.links( dist_list[distN],N )
    #пробегаем во всем квартирам в округе
    print(len(links),end=' - > ')
    filt_links = []
    for link in links:
        if ('https://www.cian.ru/sale/flat/'+str(link) ) not in parsed_links:
            filt_links.append(link)
    print(len(filt_links))
    for link in filt_links:
        print('#',end = '')
        #словарь из фич для данной квартиры
        ind_data_dic=get_data_dic.get_data_dic (link)
        #добавляем поле District, которое зависит от района
        ind_data_dic['Search_group']=dist_list_show[distN]
        ind_data_dic['ID']=ID
        ind_data_dic['Link']='http://www.cian.ru/sale/flat/' + str(link) + '/'
        ind_data_dic['parsing_date'] = time.strftime("%d_%m_%Y")
        ID+=1
        #добавляем словарь фич для квартиры в итоговый датасет    
        X.append(ind_data_dic)
    #print("preparing data")
    data=pd.DataFrame(X)
    
    #print(str(distN) + '/' + str(len(dist_list)))
    data.to_csv("output\data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") + ".csv",sep=';')
    clear_output()
data.to_csv("output\FINAL_data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") + ".csv",sep=';')
print('Done')

Local current time : (10, 44)
['http://www.cian.ru/sale/flat/152562098/', 'http://www.cian.ru/sale/flat/152568595/', 'http://www.cian.ru/sale/flat/149888462/', 'http://www.cian.ru/sale/flat/151908004/', 'http://www.cian.ru/sale/flat/152320346/', 'http://www.cian.ru/sale/flat/150314057/', 'http://www.cian.ru/sale/flat/152050630/', 'http://www.cian.ru/sale/flat/152115711/', 'http://www.cian.ru/sale/flat/152115710/', 'http://www.cian.ru/sale/flat/150753951/', 'http://www.cian.ru/sale/flat/151794500/', 'http://www.cian.ru/sale/flat/151363597/', 'http://www.cian.ru/sale/flat/151820967/', 'http://www.cian.ru/sale/flat/151798966/', 'http://www.cian.ru/sale/flat/151240474/', 'http://www.cian.ru/sale/flat/150718149/', 'http://www.cian.ru/sale/flat/151940311/', 'http://www.cian.ru/sale/flat/151163081/', 'http://www.cian.ru/sale/flat/149951113/', 'http://www.cian.ru/sale/flat/150817353/', 'http://www.cian.ru/sale/flat/151437998/', 'http://www.cian.ru/sale/flat/152119922/', 'http://www.cian.ru/sal

  0%|                                                                                           | 0/49 [00:00<?, ?it/s]

11 - > 11
#########

KeyboardInterrupt: 

['153258054',
 '153301611',
 '153279570',
 '152928993',
 '153221936',
 '147833870',
 '153317944',
 '152817181',
 '153264810',
 '153318829',
 '152707131']

In [3]:
time.strftime("%d_%m_%Y")

'28_01_2017'

In [1]:
import pith
import requests
import re
from bs4 import BeautifulSoup

111.76693048363846


In [12]:
flat_page = requests.get('https://www.cian.ru/sale/flat/152165060/')
print((str(flat_page)))


<Response [200]>


In [15]:
clear_output()
from tqdm import tqdm
def clear():
    subprocess.Popen( "cls" if platform.system() == "Windows" else "clear", shell=True)

for i in tqdm(range(10)):
    
    for j in range(10):
        
        print('#',end='')
        time.sleep(1)
    


 10%|████████▎                                                                          | 1/10 [00:10<01:30, 10.07s/it]

#########

KeyboardInterrupt: 

In [6]:
#data[['Balcony', 'Built_year', 'Description', 'Floor', 'Home_type', 'ID']]
#data[['Kitsp', 'Lift', 'Link', 'Livesp', 'Metro_station', 'Metro_time']]
#data[['Nfloor', 'Parking', 'Price', 'Remont', 'Roof_hight', 'Sales_type']]
#data[['Sdacha_GK', 'Search', 'Tel', 'Totsp', 'View', 'WC', 'Walk_metro']]
data[['dist', 'is_premium', 'lat', 'lon', 'publication_date', 'rooms','sale']]

,dist,is_premium,lat,lon,publication_date,rooms,sale
0,4.748564,"false,",55.724636,37.581748,1483786642,3,"""sale"","
1,6.945398,"false,",55.859261,37.602058,1483812362,1,"""sale"","


In [1]:
def heat_map(x,y):
    import numpy as np
    import matplotlib.pyplot as plt
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=200)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    #plt.figure(figsize=(10,5))
    plt.imshow(heatmap.T, extent=extent, origin='lower')
    plt.show()


In [21]:
def split_link(link):
    return link.split('/')[-2]
split_link('https://www.cian.ru/sale/flat/150718149/')

'150718149'

In [22]:
links

['153258054',
 '153301611',
 '153279570',
 '152928993',
 '153221936',
 '147833870',
 '153317944',
 '152817181',
 '153264810',
 '153318829',
 '152707131']